In [1]:
import pandas as pd
import numpy as np
from PIL import Image,ImageFilter
import cv2
import matplotlib.pyplot as plt
import heapq

In [86]:
# 首先看图片大小是否一样,不是很好存
with open('data/dataset/manmade_training.txt','r',encoding='utf-8') as f:
    im_name = [line.strip('\n') for line in f.readlines()]

size = np.zeros((len(im_name),2))  
cnt = 0
for path in im_name:
    path = 'data/dataset/' + path
    im = np.array(Image.open(path))
    size[cnt] = np.shape(im[:,:,0])
    cnt += 1

size



array([[1024.,  768.],
       [1290., 2032.],
       [1244.,  877.],
       [ 851., 1280.],
       [2200., 1650.],
       [ 240.,  320.],
       [ 400.,  600.],
       [ 224.,  400.],
       [ 600.,  800.],
       [1024., 1536.],
       [ 330.,  250.],
       [2304., 3072.],
       [ 480.,  640.],
       [ 545.,  741.],
       [ 275.,  204.],
       [2448., 3264.],
       [1704., 2272.],
       [ 316.,  200.],
       [ 960., 1280.],
       [ 600.,  800.],
       [ 256.,  256.],
       [ 768., 1024.],
       [ 384.,  256.],
       [ 202.,  310.],
       [2112., 2816.],
       [1577., 2221.],
       [ 300.,  224.],
       [ 256.,  256.],
       [1536., 2048.],
       [ 256.,  256.],
       [ 480.,  640.],
       [ 402.,  600.],
       [ 240.,  320.],
       [2304., 3456.],
       [ 800.,  481.],
       [ 256.,  256.],
       [2592., 1944.],
       [1368., 1712.],
       [ 549.,  732.],
       [1320.,  960.],
       [ 240.,  320.],
       [ 240.,  320.],
       [2448., 3264.],
       [ 96

In [2]:
# This function will get the pixels of each partition

def get_pixels(num_pixels_x, num_tiles_x):
    tile_pixels_x = np.zeros(3)
    if num_pixels_x % num_tiles_x == 0:
        tile_pixels_x = num_pixels_x/num_tiles_x
    else:
        tile_pixels_x[2] = int(num_pixels_x/num_tiles_x)
        remain = num_pixels_x % num_tiles_x
        if remain % 2 == 0:
            tile_pixels_x[0] = tile_pixels_x[1] + remain/2
            tile_pixels_x[2] = tile_pixels_x[1] + remain/2
        else:
            tile_pixels_x[0] = tile_pixels_x[1] + int(remain/2)
            tile_pixels_x[2] = tile_pixels_x[1] + int(remain/2) + 1

    x1 = np.zeros(num_tiles_x + 1)
    x1[1] = tile_pixels_x[0]
    x1[2:-1] = tile_pixels_x[1]
    x1[-1] = tile_pixels_x[2]

    return x1


# This method will randomly allocate the extra pixels to tiles
# return the 刻度
def get_pixels_ran(num_pixels_x, num_tiles_x):
    tile_pixels_x = np.zeros(num_tiles_x)
    if num_pixels_x % num_tiles_x == 0:
        tile_pixels_x = (num_pixels_x/num_tiles_x)*np.ones(num_tiles_x).astype(int)
    else:
        tile_pixels_x = int(num_pixels_x/num_tiles_x)*np.ones(num_tiles_x).astype(int)
        remain = num_pixels_x % num_tiles_x
        position = np.random.randint(0,num_tiles_x, size = remain)

        for i in range(remain):
            tile_pixels_x[position[i]] += 1

    cum_sum = np.cumsum(tile_pixels_x).astype(int)
    tile_pixels_x = cum_sum - int(tile_pixels_x[0])
    tile_pixels_x = np.append(tile_pixels_x,num_pixels_x)
    return tile_pixels_x
        
        


# This function will calculate the chi_square of pictures
def chi_square(X,Y,bins):
    # First Get the histogram with given bins of
    # Cost too much time 
    hist_x = np.zeros((3,bins))
    hist_y = np.zeros((3,bins))
    d = np.zeros((3,1))

    for i in range(3):#三个循环分别计算R,G,B
        hist_x[i], cnt = np.histogram(X[:,:,i], bins = bins) #ax[i,0].hist(X[:,:,i]),第三个元素为RGB参数
        #下面是归一化，变成所有和为一
        hist_x[i] = hist_x[i]/np.sum(hist_x[i])
        hist_y[i], cnt = np.histogram(Y[:,:,i], bins = bins)
        hist_y[i] = hist_y[i]/np.sum(hist_y[i])
        d[i] = np.sum(np.square(hist_x[i] - hist_y[i])/(hist_x[i] + hist_y[i]+1))

    return np.sqrt(np.sum(np.square(d)))

# This chi-square methos compares two histogram，这个没有太理解
def chi_square_hist(x,y):
    if np.shape(x) == (3,20):
        d = np.zeros((3,1))
        for i in range(3):
            d[i] = np.sum(np.square(x[i] - y[i])/(x[i] + y[i] + 1))
        result = np.sqrt(np.sum(np.square(d)))
    else :
        result = np.sum(np.square(x - y)/(x + y + 1))

    return result




# I is the original image, x and y is the output image size
def gauss_subsample(I,x,y):
    I = I.filter(ImageFilter.GaussianBlur)
    I = I.resize((x,y))
    return I 

    




In [88]:
def get_hist(image,bins):
    # 假设可以整除
    interval_len = 256/bins
    result_hist = np.zeros(bins**3)
    for i in range(np.shape(image)[0]):
        for j in range(np.shape(image)[1]):
            result = 0
            for k in range(3):
                result = result + int(image[i,j,k]/interval_len) * (bins**k)
                
            result_hist[result] += 1

    return result_hist.astype(int)

    

In [89]:
# 选取top10中质地相似的图片，只选择一张
#def make_similar_image(matrixIN , tar_matrix):
#    tar_size_width=tar_matrix.shape[1]
#    tar_size_height=tar_matrix.shape[0]
#    return np.resize(matrixIN,(tar_size_height,tar_size_width))

#def hist_similar(lh, rh):
#    assert len(lh) == len(rh)
#    return sum(1 - (0 if l == r else float(abs(l - r))/max(l, r)) for l, r in zip(lh, rh))/len(lh)

#def calc_similar(li, ri):
#    return hist_similar(li.histogram(), ri.histogram())

#def calc_similar_by_path(lf, rf):
#    li, ri = make_similar_image(Image.open(lf)), make_similar_image(Image.open(rf))
#    return calc_similar(li, ri)

#def split_image(img, part_size = (64, 64)):
#    w, h = img.size
#    pw, ph = part_size
#    assert w % pw == h % ph == 0
#    return [img.crop((i, j, i+pw, j+ph)).copy() for i in xrange(0, w, pw) for j in xrange(0, h, ph)]

#def calc_similar(li, ri):
#    #return hist_similar(li.histogram(), ri.histogram())
#    return sum(hist_similar(l.histogram(), r.histogram()) for l, r in zip(split_image(li), split_image(ri))) / 16.0


In [90]:
# 选取top10中质地相似的图片，只选择一张
# 将top10图片矩阵的大小变成和目标图像分块一样
def make_similar_image(matrixIN , tar_matrix):
    tar_size_width=np.shape(tar_matrix)[1]
    tar_size_height=np.shape(tar_matrix)[0]
    return np.resize(matrixIN,(tar_size_height,tar_size_width))

def matrix_distance(matrix1, matrix2):
    #矩阵之间的欧氏距离
    assert np.shape(matrix1) == np.shape(matrix2)
    sum=0
    [m,n]=np.shape(matrix1)
    for i in range(m):
        for j in range(n):
            sum=sum+np.square((matrix1[i,j]-matrix2[i,j]))
    sum=np.sqrt(sum)
    return sum

#def calc_similar(li, ri):
#    return hist_similar(li.histogram(), ri.histogram())

#def calc_similar_by_path(lf, rf):
#    li, ri = make_similar_image(Image.open(lf)), make_similar_image(Image.open(rf))
#    return calc_similar(li, ri)

def split_image_lists(matrixIN, split):
    #将图片对应矩阵分块，split为将长宽分别几等分
    [m,n]=np.shape(matrixIN)
    size_m=m/split
    size_n=n/split
    parts=list()
    for i in range(split):
        for j in range(split):
            part=matrixIN[i*size_m : (i+1)*size_m , j*size_n : (j+1)*size_n]
            parts.append(part)
        #parts是分割的图像矩阵一个一个排列在次列表内
    return parts

#    return [img.crop((i, j, i+pw, j+ph)).copy() for i in xrange(0, w, pw) for j in xrange(0, h, ph)]

def calc_distance(list1, list2):
    assert len(list1) == len(list2)
    n=len(list1)
    distance=0
    for i in range(n):
        dis=matrix_distance(list1[i],list2[i])
        distance=distance+dis
    #此处返回某图片与目标图片分块的距离
    return distance


In [91]:
def hist_rgb(target, result, dict_pics, tiles_x, tiles_y,x1,x2,y1,y2):
    bins = 20
    
    import heapq
    # ============================================get hist==========================================
    for key,pic in dict_pics.items():
        pic_hist = np.zeros((3,bins))
        for i in range(3):
            tmp = np.histogram(pic[:,:,i], range = (0,255) ,density =True, bins = bins)[0]
            pic_hist[i] = tmp
        # 把数据集里面的图片的直方图全部放到一个list中
        dict_pics[key] = pic_hist

    # # =======================================get target pic=================================================
    print("All histogram have been generated. Begin to generate the picture~~~~")
    #将target图像进行分块，方便之后的比对
    for i in range(tiles_y):
        for j in range(tiles_x):
            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            #使用list对heap进行初始化
            heap = []


            # ==================================Get chi distance======================================

            # ============Method1: Get the RGB distance then using average to get the best==============
            # get histogram of target pictures得到目标图像分块的直方图
            target_ij_hist = np.zeros((3,bins))
            for k in range(3):
                tmp = np.histogram(target_ij[:,:,k],range = (0,255), bins = bins, density=True)[0]
                target_ij_hist[k] = tmp   #这个是目标图像分块后的每一小块的直方图

            for item in dict_pics.items():
                # input vector shape (3,bins)
                #这里的item[0][1]是什么意思,用item[0]存储每张图片的距离，用来求最近的几张
                distance = chi_square_hist(target_ij_hist,item[1])
                dict_path[item[0]] = distance

            # # ===============Method2: Using one histogram（TOO SLOW!）=================
            # # Create the hist
            # target_ij_hist2 = get_hist(target,8)

            # # Get the hist of pics
            # for item in dict_pics.items():
            #     dict_path[item[0]] = chi_square_hist(target_ij_hist2,item[1])
            

            # ========================================Get top k===============================================
            #取出了前十个
            top = heapq.nsmallest(10,dict_path.items(), key = lambda x : x[1])
            #下面第一个list用来存放十张top图片与目标分块图片的距离
            distance_list=list()
            tar_list=split_image_lists(target_ij,4)
            for t in range(len(top)):
                tmp=dict_pics[top[t][0]]
                tmp=make_similar_image(tmp,target_ij)
                tmp_list=split_image_lists(tmp,4)
                distance=calc_distance(tar_list,tmp_list)
                distance_list.append(distance)
            bstDistance=heapq.nsmallest(1,distance)
            index=distance.index(bstDistance)
            trueTop=top[index]
                
#1.这里看一下，改成提取前十个，然后再自己想一些特征去判断，这个是判断texture
#2.判断target图像的颜色特征后，将图像集进行调整，缩小图像集的大小，使最后呈现的图片不会很乱
            # print("Get the patch of %s, %s" % (i,j))
            # print(trueTop)
            # =================================From topk to get the top 1=====================================
#这里加一点，就加入再分块比较的思想（上面取的还是前十位）
            # ==============================Process the best image into proper size===========================
            # Then fit the image into the proper size: Using Gaussian subsample
            # First get the Gaussian smoothing
            # Then get the subsample with the proper size
            im = Image.open(trueTop[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

    return result


In [92]:
def cosin(target, result,dict_pics, tiles_x, tiles_y,x1,x2,y1,y2,size):
    print("I entered the cosin method!!!!~")
    # ===========================将图片标准化========================
    def get_thum(image, size=(size, size), greyscale=False):
    # 利用image对图像大小重新设置, Image.ANTIALIAS为高质量的
        image = Image.fromarray(image)
        image = image.resize(size, Image.ANTIALIAS)
        if greyscale:
            # 将图片转换为P模式，其每个像素用8个bit表示
            image = image.convert('P')
        return image


    def get_cosin(image1,pic_array):
        image1_array = np.array(get_thum(image1)).flatten()
        return np.dot(image1_array,pic_array.T)/(np.linalg.norm(image1_array)*np.linalg.norm(pic_array))
        


    # 首先将1000张图片进行一个压缩，变成灰度图像
    for key, pic in dict_pics.items():
        dict_pics[key] = np.array(get_thum(pic)).flatten()

    print("All pictures have been converted to 'P'. Start to generate picture!!!!~")
    for i in range(tiles_y):
        for j in range(tiles_x):
            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []
            # =============================获得余弦值==============================
            for key, pic_array in dict_pics.items():
                dict_path[key] = get_cosin(target_ij,pic_array)
            
            # ============================得到top===========================
            top = heapq.nlargest(1,dict_path.items(), key = lambda x:x[1])
            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

    return result




In [93]:
# First give the picture different color category 

### Using hash to get the similar picture

In [94]:
def ahash_compare(target, result,dict_pics, tiles_x, tiles_y,x1,x2,y1,y2):
    print("I haved entered ahash_compare methos!!!~~")
    
    def ahash(image):
        # 将图片缩放为8*8的
        image = Image.fromarray(image)
        #image = cv2.resize(image, (8, 8), interpolation=cv2.INTER_CUBIC)
        image = image.resize((8, 8))
        # 将图片转化为灰度图
        #gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        gray = np.array(image.convert('L'))
        # s为像素和初始灰度值，hash_str为哈希值初始值
        s = 0
        # 遍历像素累加和
        for i in range(8):
            for j in range(8):
                s = s + gray[i, j]
        # 计算像素平均值
        avg = s / 64
        # 灰度大于平均值为1相反为0，得到图片的平均哈希值，此时得到的hash值为64位的01字符串
        ahash_str = ''
        for i in range(8):
            for j in range(8):
                if gray[i, j] > avg:
                    ahash_str = ahash_str + '1'
                else:
                    ahash_str = ahash_str + '0'
        result = ''
        for i in range(0, 64, 4):
            result += ''.join('%x' % int(ahash_str[i: i + 4], 2))
        # print("ahash值：",result)
        return result

    # phash
    def phash(image):
        # 加载并调整图片为32*32的灰度图片
        # img = cv2.imread(path)
        # img1 = cv2.resize(img, (32, 32),cv2.COLOR_RGB2GRAY)
        image = Image.fromarray(image)
        image = image.resize((32,32))
        image = image.convert('L')
        
        # 创建二维列表
        h, w = image.shape[:2]
        vis0 = np.zeros((h, w), np.float32)
        vis0[:h, :w] = image

        # DCT二维变换
        # 离散余弦变换，得到dct系数矩阵
        img_dct = cv2.dct(cv2.dct(vis0))
        img_dct.resize(8,8)
        # 把list变成一维list
        img_list = np.array().flatten(img_dct.tolist())
        # 计算均值
        img_mean = cv2.mean(img_list)
        avg_list = ['0' if i<img_mean else '1' for i in img_list]
        return ''.join(['%x' % int(''.join(avg_list[x:x+4]),2) for x in range(0,64,4)])

    # 差异值哈希算法
    def dhash(image):
        # 将图片转化为8*8
        image = Image.fromarray(image)
        image = image.resize((9,8))
        gray = np.array(image.convert('L'))

        # image = cv2.resize(image, (9, 8), interpolation=cv2.INTER_CUBIC)
        # # 将图片转化为灰度图
        # gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        dhash_str = ''
        for m in range(8):
            for n in range(8):
                if gray[m , n] > gray[m, n + 1]:
                    dhash_str = dhash_str + '1'
                else:
                    dhash_str = dhash_str + '0'
        result = ''
        for m in range(0, 64, 4):
            result += ''.join('%x' % int(dhash_str[m: m + 4], 2))
        # print("dhash值",result)
        return result


    # 计算两个哈希值之间的差异
    def compHash(hash1, hash2):
        n = 0
        # hash长度不同返回-1,此时不能比较
        if len(hash1) != len(hash2):
            return -1
        # 如果hash长度相同遍历长度
        for i in range(len(hash1)):
            if hash1[i] != hash2[i]:
                n = n + 1
        return n

    # =========首先得到1000张图片的ahash==========
    # for key, pic in dict_pics.items():
    #     dict_pics[key] = ahash(pic)
    for key, pic in dict_pics.items():
        dict_pics[key] = ahash(pic)
    
    # =========开始检索========
    print("All hash code have been obtained! Begin to generate!!!~")
    for i in range(tiles_y):
        for j in range(tiles_x):
            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []
             
            for key, hash_value in dict_pics.items():
                dict_path[key] = compHash(ahash(target_ij),hash_value)

            top = heapq.nsmallest(1,dict_path.items(), key = lambda x:x[1])

            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)

    return result
 

In [95]:
def lab_compare(target, result,dict_pics, tiles_x, tiles_y,x1,x2,y1,y2):
    # calculate lab color distance
    def calculate_color_distance(color1, color2):
        rmean = (int(color1[0]) - int(color2[0]))/2
        r = int(color1[0]) - int(color2[0])
        g = int(color1[1]) - int(color2[1])
        b = int(color1[2]) - int(color2[2])
        distance = np.sqrt(((2+rmean/256)*r*r + 4*g*g + (2+(255-rmean)/256)*b*b))
        return distance

    # First resize the pics into corresponding size of 得到每张分块图片的大小
    x = int(np.shape(target)[1]/tiles_x)
    y = int(np.shape(target)[0]/tiles_y)

    # Then resize pics dataset into this size将图片变成需要的大小
    for key, pics in dict_pics.items():
        tmp = Image.fromarray(pics)
        tmp = np.array(tmp.resize((x,y)))
        dict_pics[key] = tmp

    # begin to compare each pixels, 直接忽略边缘
    print("All pictures have been resized into proper size! Begin to generate!!!~")
    for i in range(tiles_y):
        for j in range(tiles_x):
            # Get target part of pic
            target_ij = target[y1[i]:y1[i+1],x1[j]:x1[j+1]]
            y_pixels = y2[i+1] - y2[i]
            x_pixels = x2[j+1] - x2[j]

            # compare target_ij with the image
            # First just read one then compare one and get the top 10
            dict_path = {}
            heap = []

            # compute the distance of each pixels
            # 这个地方或许可以优化？
            
            for key, pics in dict_pics.items():
                sum_lab = 0
                for m in range(y):
                    for n in range(x):
                        sum_lab += calculate_color_distance(target_ij[m,n],pics[m,n])
                
                dict_path[key] = sum_lab

            # get the smallest
            top = heapq.nsmallest(1,dict_path.items(), key = lambda x:x[1])
            
            #top = heapq.nsmallest(10,dict_path.items(), key = lambda x:x[1])
            #for bst in top
#1.这里看一下，改成提取前十个，然后再自己想一些特征去判断，这个是判断texture
#2.判断target图像的颜色特征后，将图像集进行调整，缩小图像集的大小，使最后呈现的图片不会很乱
            im = Image.open(top[0][0])
            im = gauss_subsample(im,x = x_pixels, y = y_pixels)

            # 向结果矩阵中拼接
            result[y2[i]: y2[i+1], x2[j]: x2[j+1]] = np.array(im)
        
    return result
                
                    
    
    

### Start to generate the pictures

In [96]:
def main(target,pixels_x, pixels_y, tiles_x,tiles_y):
    # first get the partition of original pic
    # x1,y1 表示原始图像中的分割
    # x2,y2 表示输出图像中的分割大小，找到对应图片后应该压缩到的大小
    bins = 20
    x1 = get_pixels_ran(np.shape(target[:,:,0])[1], tiles_x).astype(int)
    y1 = get_pixels_ran(np.shape(target[:,:,0])[0], tiles_y).astype(int)

    # construct x and y with the size of tiles
    x2 = get_pixels_ran(pixels_x, tiles_x).astype(int)
    y2 = get_pixels_ran(pixels_y, tiles_y) .astype(int)

    # result image
    result = np.zeros((pixels_y, pixels_x, 3))

    # ===========================获得每张图片==========================
    # -------------纯色图片测试集------------
    #list_color = ['#47493c','#cdcaa8','#494630','#7c95ac','#7c95ac','#7c95ac','#506b8b','#19180e','#748570','#241b14','#b48c2c']
    #dictMerged = {}
    #for color in list_color:
    #    path = 'data/pure_color/' + color + '.jpg'
    #    im = Image.open(path)
    #    im = np.array(im)
    #    dictMerged[path] = im
        
    # #########1000张training图片集########
    #  将所有图片读成两个数组：man/natural

    with open('data/dataset/manmade_training.txt','r',encoding='utf-8') as f:
         im_name_m = [line.strip('\n') for line in f.readlines()]
    with open('data/dataset/natural_training.txt','r',encoding='utf-8') as f:
         im_name_n = [line.strip('\n') for line in f.readlines()]

    

     # 将图片存储字典中
    dict_images_m = {}
    dict_images_n = {}
    for path in im_name_m:
         path = 'data/dataset/' + path
         im = Image.open(path)
         im = np.array(im)
         dict_images_m[path] = im

    for path in im_name_n:
         path = 'data/dataset/' + path
         im = Image.open(path)
         im = np.array(im)
         dict_images_n[path] = im

    dictMerged = dict( dict_images_m, **dict_images_n )
    
    # ==============================Using different method to find the result===========================
    result = hist_rgb(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2)  

    # size = min(int(np.shape(target[:,:,0])[1]/tiles_x),int(np.shape(target[:,:,0])[0]/tiles_y))
    # result = cosin(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2,size)
          
    # result = ahash_compare(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2) 
    
    #result = lab_compare(target,result,dictMerged,tiles_x,tiles_y,x1,x2,y1,y2) 
    return result



            

In [97]:
def store_result(path,num,pixels_x,pixels_y, tile_x,tile_y):
    result = main(np.array(Image.open(path)), pixels_x,pixels_y,tile_x,tile_y)
    Image.fromarray(np.uint8(result)).save('results/results%s/lab_para(%s,%s).jpg' % (num,pixels_x,tile_x))

In [98]:
store_result('data/target/target1.jpg',1, 1750,1250,75,25)

MemoryError: Unable to allocate 22.8 MiB for an array with shape (2304, 3456, 3) and data type uint8

### Script

In [ ]:
# test chi
# I1 = Image.open('data/target/target1.jpg') 
# I2 = Image.open('data/target/target2.jpg')
# I1 = np.array(I1)
# I2 = np.array(I2)
# chi_square(I1,I1,20)

# test random 
get_pixels_ran(1698,3)


[566. 566. 566.]


array([   0,  566, 1132, 1698])

In [ ]:
I = np.array(Image.open('data/target/target1.jpg'))
Image.fromarray(I[0:4,0:4])


In [ ]:
dict_test = {}
dict_test['a'] = 4
dict_test['b'] = 2
dict_test['c'] = 3
dict_test['0'] = 5
a = heapq.nsmallest(1,dict_test.items(), key = lambda x : x[1])
a[0][0]

'b'

In [ ]:
for key, value in dict_test.items():
    print(key,value)

a 4
b 2
c 3
0 5


In [ ]:
for item in dict_test.items():
    print(item[0])

a


In [ ]:
get_hist(I,8)

array([ 67067,  18597,      0,      0,      0,      0,      0,      0,
         4137, 165236,   7301,      1,      0,      0,      0,      0,
            0,   8654,  38095,   2050,      4,      0,      0,      0,
            0,      0,   3919,  14869,    714,      2,      0,      0,
            0,      0,      0,   2476,   5438,    129,      0,      0,
            0,      0,      0,      0,    772,    367,      0,      0,
            0,      0,      0,      0,      0,     75,      1,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
          509,   1153,      0,      0,      0,      0,      0,      0,
          257, 136243,  45948,     15,      0,      0,      0,      0,
            0,   2955, 130935,   9080,      2,      0,      0,      0,
            0,      0,   2236,  24671,   2827,      4,      0,      0,
            0,      0,      0,   3586,  20843,   2165,     15,      0,
            0,      0,      0,      0,   5395,  18169,    814,      0,
      

In [ ]:
chi_square_hist(get_hist(I,8),get_hist(I,8))

0.0

In [ ]:
chi_square_hist(np.histogram(I[:,:,0],20)[0],np.histogram(I[:,:,0],20)[0])

0.0

In [ ]:
list_color = ['#47493c','#cdcaa8','#494630','#7c95ac','#7c95ac','#7c95ac','#506b8b','#19180e','#748570','#241b14','#b48c2c']
path = 'data/pure_color/' + '#cdcaa8' + '.jpg'
im = Image.open(path)
im = np.array(im)
np.histogram(im[:,:,0], range = (0,255) ,density=True, bins = 20)[0]



array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.07843137, 0.        , 0.        , 0.        ])